In [58]:
# !pip install --upgrade implicit

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [36]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [37]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [38]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [39]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [40]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id']
    weights = items_weights['weight']
    
    # передаем дополнительно веса
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [41]:
# подготовим веса по объему продаж
def get_weighted_polpular(data_in, n_popular=0):
    popular = data_in.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    if n_popular > 0:
        popular = popular[:n_popular]

    # посолим и прологарифмируем
    popular['weight'] = popular['sales_value'].apply(lambda x: np.log(x+1e-5))
    # приведем мин в 0 (сделаем неотрицательными)
    popular['weight'] = popular['weight'] - popular['weight'].min()  

    log_sum = popular['weight'].sum()
    # найдем веса
    popular['weight'] = popular['weight']/log_sum

    weighted_popular = popular[['item_id', 'weight']].copy()
    print(weighted_popular['weight'].sum())
    return weighted_popular


In [42]:
w_popular = get_weighted_polpular(data_train)

1.0


**тест взвешенных рекомендаций**

In [43]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(w_popular, n=5))
result.head(5)


CPU times: user 4.43 s, sys: 0 ns, total: 4.43 s
Wall time: 4.43 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1061364, 12263239, 9487898, 1010172, 14077294]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9575413, 882866, 10254962, 903674, 10204548]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5589657, 1155854, 1957702, 6391302, 6772652]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[933195, 10456466, 1008799, 12385198, 642430]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2474817, 74230, 963664, 12757356, 9837093]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [44]:
pred_basic = pd.read_csv('./data/predictions_basic.csv')
pred_basic.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


**Precision@5**

In [45]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    def _str_to_list(in_str):
        return in_str[1:-1].split(',')

    if type(recommended_list) is type('str'):
        recommended_list = _str_to_list(recommended_list)
        
    if type(bought_list) is type('str'):
        bought_list = _str_to_list(bought_list)

    # непосредственно расчет    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

**функция тестирование датасета**

In [46]:
def test_prediction_at_k(data_pr, actual_idx=1, start_idx=2):
    actual_col = data_pr.columns[actual_idx]
    recomend_algo_cols = data_pr.columns[start_idx:]

    algos_means = []
    for algo in recomend_algo_cols:
        algo_mean = data_pr.apply(lambda row: precision_at_k(row[algo], row[actual_col]), axis=1).mean()
        algos_means.append(algo_mean)
        print(f'Алгоритм: {algo}, mean = {algo_mean}')

    max_idx = np.asarray(algos_means).argmax()
    print()
    print(f'Максимальный результат у {recomend_algo_cols[max_idx]} = {algos_means[max_idx]}')

In [47]:
%%time
# тестирование предоставленного датасета
test_prediction_at_k(pred_basic)

Алгоритм: random_recommendation, mean = 0.0
Алгоритм: popular_recommendation, mean = 0.0010773751224289912
Алгоритм: itemitem, mean = 0.0
Алгоритм: cosine, mean = 0.0
Алгоритм: tfidf, mean = 0.0
Алгоритм: own_purchases, mean = 0.0

Максимальный результат у popular_recommendation = 0.0010773751224289912
CPU times: user 2.81 s, sys: 134 ms, total: 2.94 s
Wall time: 2.71 s


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code